# Equity Flows: Concentration and Distribution Snapshot

This notebook shows a compact equity flow workflow focused on concentration signals.

## What this notebook teaches
- Shared FinUties authentication setup
- Value-column normalization for ranked holdings
- Concentration diagnostics using top-share metrics

In [ ]:
from pathlib import Path
import os

import matplotlib.pyplot as plt
import pandas as pd
import requests
from dotenv import load_dotenv


def resolve_notebooks_env(start_dir: Path) -> Path:
    current = start_dir.resolve()
    for candidate_root in [current, *current.parents]:
        if candidate_root.name == "notebooks":
            env_path = candidate_root / ".env"
            if env_path.exists():
                return env_path
        nested_env = candidate_root / "notebooks" / ".env"
        if nested_env.exists():
            return nested_env
    raise FileNotFoundError(
        "Missing notebooks/.env. Create it from notebooks/.env.example and set FINUTIES_API_KEY."
    )


load_dotenv(resolve_notebooks_env(Path.cwd()))
API_ORIGIN = os.getenv("FINUTIES_API_ORIGIN", "https://data.finuties.com").rstrip("/")
API_KEY = os.getenv("FINUTIES_API_KEY", "").strip()
if not API_KEY:
    raise ValueError("Missing FINUTIES_API_KEY in notebooks/.env")
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

In [ ]:
response = requests.get(
    f"{API_ORIGIN}/api/v1/holdings/top-companies",
    headers=HEADERS,
    params={"limit": 200},
    timeout=30,
)
response.raise_for_status()
payload = response.json()
rows = payload if isinstance(payload, list) else payload.get("items") or payload.get("data") or []
df = pd.DataFrame(rows)
if df.empty:
    raise ValueError("No equity flow rows returned")

value_col = "market_value" if "market_value" in df.columns else ("value" if "value" in df.columns else None)
if not value_col:
    raise ValueError(f"No recognized value column. Available columns: {list(df.columns)}")

In [ ]:
df[value_col] = pd.to_numeric(df[value_col], errors="coerce").fillna(0.0)
df = df[df[value_col] > 0].copy()

top = df.sort_values(value_col, ascending=False).head(15).reset_index(drop=True)
total_value = top[value_col].sum()
top["share_pct"] = (top[value_col] / total_value * 100).round(2)
concentration_top5 = top["share_pct"].head(5).sum()

display_cols = [c for c in ["company", "symbol", value_col, "share_pct"] if c in top.columns]
print(f"Top-5 concentration (percent of shown basket): {concentration_top5:.2f}%")
top[display_cols]

In [ ]:
labels = top["symbol"] if "symbol" in top.columns else top.index.astype(str)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].bar(labels.astype(str), top[value_col], color="#1f77b4")
axes[0].set_title("Top Holdings by Value")
axes[0].set_xlabel("Symbol")
axes[0].set_ylabel("Value (USD)")
axes[0].tick_params(axis="x", rotation=45)

axes[1].bar(labels.astype(str), top["share_pct"], color="#9467bd")
axes[1].set_title("Relative Share of Top Basket")
axes[1].set_xlabel("Symbol")
axes[1].set_ylabel("Share (%)")
axes[1].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

## Interpretation and caveats

- High top-5 concentration can indicate crowding risk in allocation.
- This snapshot is endpoint-dependent; always verify coverage and universe definitions.

## Community extension ideas

- Track concentration over time by saving daily snapshots.
- Compare concentration across sectors or investor cohorts when available.